In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
import arviz as az
import pymc as pm
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cellbayesassay as cba
import attila_utils

In [2]:
cba.get_TI_conc()

Study    TI       conc
CO28151  TI1      c1      1.000000e-04
                  c2      1.000000e-05
                  c3      1.000000e-06
         TI2      c1      1.000000e-04
                  c2      1.000000e-05
                  c3      1.000000e-06
         TI3      c1      1.000000e-04
                  c2      1.000000e-05
                  c3      1.000000e-06
         TI4      c1      1.000000e-04
                  c2      1.000000e-05
                  c3      1.000000e-06
         TI5      c1      1.000000e-04
                  c2      1.000000e-05
                  c3      1.000000e-06
         TI6      c1      1.000000e-05
                  c2      1.000000e-06
                  c3      2.000000e-07
         TI7      c1      1.000000e-04
                  c2      1.000000e-05
                  c3      1.000000e-06
         TI8      c1      1.000000e-04
                  c2      1.000000e-05
                  c3      1.000000e-06
         TI9      c1      1.000000e-04
  

In [3]:
cba.get_TI_name()

Name
Study   TI                                               
CO28151 TI1                                     Amiloride
        TI2                Phenamil methanesulfonate salt
        TI3                Benzamil hydrochloride hydrate
        TI4       5-(N,N-Dimethyl)amiloride hydrochloride
        TI5              5-(N-Ethyl-N-isopropyl)amiloride
        TI6                5-(N,N-Hexamethylene)amiloride
        TI7              5-(N-Methyl-N-isobutyl)amiloride
        TI8                                        NS 383
        TI9                                      A-317567
        TI10                                   Cilostazol
        TI11                                DIM-C-pPhOCH3
        TI12                                      C-DIM12
        TI13                                  Liraglutide
        TI14                                  Semaglutide
        TI15                   Hydroxychloroquine racemic
CO28153 TI16/17  (S)Hydroxychloroquine racemic, free base
        TI18/19  (R)Hydroxychloroquine racemic, free base

In [4]:
%connect_info

{
  "shell_port": 60319,
  "iopub_port": 60320,
  "stdin_port": 60321,
  "control_port": 60323,
  "hb_port": 60322,
  "ip": "127.0.0.1",
  "key": "3ff8b717-67664ccf16afc03651c6e488",
  "transport": "tcp",
  "signature_scheme": "hmac-sha256",
  "kernel_name": "python3",
  "jupyter_session": "/Users/jonesa7/CTNS/notebooks/2024-02-14-cell-bayes/2024-02-14-cell-bayes.ipynb"
}

Paste the above JSON into a file, and connect with:
    $> jupyter <app> --existing <file>
or, if you are local, you can connect with just:
    $> jupyter <app> --existing kernel-c3e1a0b3-b183-4f1a-aeaf-49eb2106ef8c.json
or even just:
    $> jupyter <app> --existing
if this is the most recent Jupyter kernel you have started.
